# NOC processing
#### This is the fourth and final step of the enrichment calculation
Calculate NOC proportion and append to the enrichment table  

Key output files
- `yyyy-mm-dd_enrichment_table_NOC_prop.csv`: enrichment values and NOC proportion  
- `yyyy-mm-dd_volcano_table.csv`: enrichment values and p-values  
  
both files are in the directory `output/enrichment_and_volcano_tables`

In [21]:
import sys
from pathlib import Path
import pandas as pd

script_path = Path.cwd().parent.parent.parent.parent / "script"
data_path = Path.cwd().parent.parent.parent.parent / "data"
sys.path.append(str(script_path))

from pyseus import basic_processing as ip
from pyseus import contrast_tools as ct

### Load imputed NOC/Bulk table
The correct datestamp is required to find the input files

In [22]:
%store -r fig5_timestamp FIG5_USE_FROZEN
if FIG5_USE_FROZEN:
    raise Exception("USE_FROZEN is true, you probably want to skip enrichment and proceed from 3.aligned_umap")
timestamp = fig5_timestamp
print(f"Timestamp: {timestamp}")

outprefix = f"{timestamp}_QC_filter_impute"

outdir = Path.cwd() / "output"
preprocessing_out = outdir / f"preprocessing"

NOC_path = preprocessing_out / f"{outprefix}_filtered_table.csv"

try:
    noc_table = pd.read_csv(NOC_path, header=[0], index_col=0)
except FileNotFoundError:
    print(f"File {NOC_path} not found.\nPlease rerun the previous steps or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {NOC_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Timestamp: 2024-10-27


### Calculation of proportion

In [23]:
# take only the noc columns
NOC_kw = ["Cytosol", "Nuclear", "Organelle"]
noc_cols = [
    col for col in list(noc_table)
    if (any(kw in col for kw in NOC_kw)) and "Proteome" not in col
]
print(noc_cols)

['UnInfected_Cytosol_1', 'UnInfected_Cytosol_2', 'UnInfected_Cytosol_3', 'UnInfected_Nuclear_1', 'UnInfected_Nuclear_2', 'UnInfected_Nuclear_3', 'UnInfected_Organelle_1', 'UnInfected_Organelle_2', 'UnInfected_Organelle_3']


In [24]:
# select the noc_cols + Protein IDs for merge purpose
noc_table = noc_table[['Protein IDs'] + noc_cols].copy()

In [25]:
# group the tables and take the median

# start RawTables class from basic processing to group the samples
noc_process = ip.RawTables(
    proteingroup=None, file_designated=True,
    info_cols=["Protein IDs"], sample_cols=noc_cols,
)

# designate the pre-computed noc filtered table on this class
noc_process.filtered_table = noc_table

# group replicates
noc_process.group_replicates(reg_exp=r"(.*)_\d+$")

# calculate the median
noc_medians = ip.median_replicates(noc_process.grouped_table)
noc_medians.head()

Intensity values have not been transformed yet from filtered table,
we recommend using transform_intensities() method before grouping replicates.

Using filtered_table to group replicates.


,UnInfected_Organelle,UnInfected_Cytosol,UnInfected_Nuclear,Protein IDs
2,76811000.0,75060000.0,166520000.0,A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124
3,194700000.0,245940000.0,238020000.0,Q9Y5S9;A0A023T787;A0A0J9YW13
5,51652000.0,4330700.0,22978000.0,A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...
6,0.0,3272600.0,0.0,A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...
7,77290000.0,9258400.0,24210000.0,Q99805;A0A024QYR8;B3KSG9


In [26]:
# we take the sum of each column 
noc_medians['noc_sum_uninf'] = noc_medians['UnInfected_Cytosol'] + noc_medians['UnInfected_Organelle'] + noc_medians['UnInfected_Nuclear']

# remove all proteingroups where both inf and uninf noc_sum is 0 -> there is no information on these
noc_medians = noc_medians[(noc_medians['noc_sum_uninf'] > 0)].reset_index(drop=True)

# create a new dataframe with just the proportion: intensity divided by the sum
noc_proportion = noc_medians[['Protein IDs']].copy()
noc_proportion['NOC_cytosol_UnInfected'] = noc_medians['UnInfected_Cytosol'] / noc_medians['noc_sum_uninf']
noc_proportion['NOC_organelle_UnInfected'] = noc_medians['UnInfected_Organelle'] / noc_medians['noc_sum_uninf']
noc_proportion['NOC_nuclear_UnInfected'] = noc_medians['UnInfected_Nuclear'] / noc_medians['noc_sum_uninf']


### Merge NOC proportion with IP enrichment

In [27]:
# read the enrichment table
enrich_out_dir = outdir / "enrichment_and_volcano_tables"
enrichment_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table.csv"
enrichments = pd.read_csv(enrichment_csv_path, header=[0, 1], index_col=0)
meta_cols = enrichments["metadata"].columns
enrichments = enrichments.droplevel(0, axis=1)

In [28]:
# finally, merge with the enrichment table
enrichment_nocs = enrichments.merge(noc_proportion, on="Protein IDs", how="inner")

# standard formatting
enrichment_nocs = ct.standard_pyseus_headers(enrichment_nocs, meta_cols=meta_cols)

In [29]:
# save the enrichment table with NOC proportions
enrichment_NOC_csv_path = enrich_out_dir / f"{timestamp}_uninf_enrichment_table_NOC_prop.csv"
enrichment_nocs.to_csv(enrichment_NOC_csv_path)

In [30]:
# enrichment table with NOC proportions
enrichment_nocs

metadata  \
                                            Protein IDs   
0         A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124   
1                          Q9Y5S9;A0A023T787;A0A0J9YW13   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...   
4                              Q99805;A0A024QYR8;B3KSG9   
...                                                 ...   
8532                                             X5D7P8   
8533                                             X5D8X9   
8534  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...   
8535                                             X5DQZ7   
8536                                             X6RHX1   

                                                                          \
                                   Majority protein IDs       Gene names   
0                       A0A023T6R1;Q96A72;F5H6P7;F5H6N1  FLJ10292;MAGOHB   
1                                     Q9Y5S9;A0A023T787       RBM8A;RBM8   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6ZTK5           TM9SF4   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...             PTEN   
4                              Q99805;A0A024QYR8;B3KSG9           TM9SF2   
...                                                 ...              ...   
8532                                             X5D7P8            STK39   
8533                                             X5D8X9          CNTNAP2   
8534  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...            CRMP1   
8535                                             X5DQZ7             GPX1   
8536                                             X6RHX1            SCYL3   

        sample                                                                \
      12-YWHAQ 14-GOLGA2     14-WT  14-RAB7A  17-RPL36 17-MAP1LC3B  17-G3BP1   
0    -2.365154  1.903443  1.492111  0.410130  0.131175    3.167371 -0.844879   
1     0.844500  0.023500  0.249350 -1.191250 -0.067500    0.131400  0.825300   
2    -5.392200  4.584300  1.166750  2.824950 -1.936000    4.301950 -1.202100   
3     0.292543  0.061496  0.554866 -0.403786 -0.889107   -0.557692 -1.011447   
4     0.442400  2.469750 -0.580900  2.383950 -2.063500    3.002300 -1.847700   
...        ...       ...       ...       ...       ...         ...       ...   
8532  0.553451  0.017426  0.560006  0.217108 -0.581026    0.236772 -0.177288   
8533 -2.114554  2.865008  2.535765  1.996572 -2.510131   -2.170080 -3.362914   
8534 -0.498195 -0.335685  0.592048 -0.029638  0.191383   -0.969134 -0.248104   
8535 -1.167690  2.563687  0.173506  2.126644 -0.268470    0.031190 -0.335495   
8536  0.618406 -0.836340  0.005384 -0.407720 -0.544768   -0.119195 -0.089789   

      ...                                                              \
      ...     10-WT   10-RTN4  09-PSMB7 11-CEP350   11-EEA1     09-WT   
0     ...  2.873520 -0.550241 -0.999255 -1.663909  0.169378  2.185520   
1     ...  1.972000 -1.899050 -0.327100  0.895300 -1.618800  1.836900   
2     ...  0.006200  1.852200 -4.414473  0.397300 -1.672800  0.315600   
3     ... -0.359181 -0.353914 -0.007011  0.117718 -0.425096  0.214107   
4     ... -0.608600  1.524950 -1.587400 -0.438800  2.055000 -0.744500   
...   ...       ...       ...       ...       ...       ...       ...   
8532  ...  0.287963 -0.071477  0.206176 -0.027301  0.200878  0.202897   
8533  ... -0.415994 -0.021098 -0.591036  1.924365  1.081366 -0.351450   
8534  ...  0.249164 -0.678933  0.670191  1.661657  1.424154  0.051400   
8535  ... -0.797759  0.706188 -0.409023 -0.045873 -0.304209 -0.219237   
8536  ...  0.246881  1.188078  0.286489 -0.630087 -0.478265 -0.023700   

                                                                \
       09-EDC4 NOC_cytosol_UnInfected NOC_organelle_UnInfected   
0     2.286857               0.235748                 0.241247   
1     1.614550               0.362391                 0.286889   
2    -0.149000               0.0548